In [1]:
import pandas as pd
import numpy as np
v1 =  pd.read_csv('dd_top_matched.csv')
v1 = v1.dropna(axis=0)
v1.corr("pearson")

,Unnamed: 0,lat,long,measured,tape,laser
Unnamed: 0,1.000000,-0.059830,-0.165065,0.035659,0.013479,0.007013
lat,-0.059830,1.000000,-0.591205,0.014794,0.018883,0.018808
long,-0.165065,-0.591205,1.000000,-0.165486,-0.164852,-0.162207
measured,0.035659,0.014794,-0.165486,1.000000,0.891040,0.904285
tape,0.013479,0.018883,-0.164852,0.891040,1.000000,0.976315
laser,0.007013,0.018808,-0.162207,0.904285,0.976315,1.000000


In [2]:
v2 = pd.read_csv('dd_results_compile.csv')
v2 = v2.dropna(axis=0)
v2.corr("pearson")

,Unnamed: 0,lat,long,tape,laser,arif_lidar,dickson_lidar
Unnamed: 0,1.000000,-0.367334,-0.719669,0.134133,0.132920,0.019469,0.056630
lat,-0.367334,1.000000,-0.308599,-0.093347,-0.077245,-0.065905,-0.130519
long,-0.719669,-0.308599,1.000000,-0.038691,-0.043646,0.047094,0.053911
tape,0.134133,-0.093347,-0.038691,1.000000,0.967312,0.857678,0.841828
laser,0.132920,-0.077245,-0.043646,0.967312,1.000000,0.843994,0.837872
arif_lidar,0.019469,-0.065905,0.047094,0.857678,0.843994,1.000000,0.825760
dickson_lidar,0.056630,-0.130519,0.053911,0.841828,0.837872,0.825760,1.000000


In [3]:
arr_less = np.where(v2['dickson_lidar']>=v2.tape)
arr_more = np.where(v2.dickson_lidar <= v2.tape)

print(f"Dickson\nOvershoot: {len(arr_more[0])}\nUndershoot: {len(arr_less[0])}")

arr_less = np.where(v2.arif_lidar >=v2.tape)
arr_more = np.where(v2.arif_lidar <= v2.tape)

print(f"Arif\nOvershoot: {len(arr_more[0])}\nUndershoot: {len(arr_less[0])}")

Dickson
Overshoot: 204
Undershoot: 37
Arif
Overshoot: 181
Undershoot: 63


In [4]:
v3 =  pd.read_csv('dd_results_compile_xy.csv')
#v3.corr("pearson")
v3 = v3.dropna(axis=0)
v3.corr("pearson")

,Unnamed: 0,lat,long,tape,laser,dickson_lidar,x,y,arif_lidar
Unnamed: 0,1.000000,-0.470928,-0.679527,0.186078,0.190679,0.096899,0.614845,0.700960,0.059540
lat,-0.470928,1.000000,-0.299451,-0.084664,-0.067567,-0.125980,-0.189368,-0.361995,-0.056460
long,-0.679527,-0.299451,1.000000,-0.131012,-0.146614,0.001273,-0.487684,-0.467095,-0.008436
tape,0.186078,-0.084664,-0.131012,1.000000,0.965917,0.829387,0.128967,0.242524,0.848425
laser,0.190679,-0.067567,-0.146614,0.965917,1.000000,0.825708,0.104414,0.270639,0.832773
dickson_lidar,0.096899,-0.125980,0.001273,0.829387,0.825708,1.000000,0.108249,0.169451,0.811895
x,0.614845,-0.189368,-0.487684,0.128967,0.104414,0.108249,1.000000,0.283533,0.080957
y,0.700960,-0.361995,-0.467095,0.242524,0.270639,0.169451,0.283533,1.000000,0.133664
arif_lidar,0.059540,-0.056460,-0.008436,0.848425,0.832773,0.811895,0.080957,0.133664,1.000000


In [5]:
import open3d as o3d
import yaml
import os
from utility.pcd2img import pcd2img_np
from utility.csf_py import csf_py
from utility.get_coords import scale_coord, draw_coord_on_img
import cv2

with open("config/config.yaml", 'r') as ymlfile:
    yml_data = yaml.load(ymlfile, Loader = yaml.FullLoader)
    
curr_dir = os.getcwd()
folder_loc = yml_data["dataset"]["folder_location"]
pcd_filename = yml_data["dataset"]["pcd"]["file_name"] + yml_data["dataset"]["pcd"]["file_type"]

pcd = o3d.io.read_point_cloud(folder_loc+pcd_filename)

non_grd = csf_py(
    pcd, 
    return_non_ground = "non_ground", 
    bsloopSmooth = True, 
    cloth_res = 15.0, 
    threshold= 2.0, 
    rigidness=1
)

#del pcd, non_grd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[0] Configuring terrain...
[0] Configuring cloth...
[0]  - width: 19 height: 19
[0] Rasterizing...
[0] Simulating...
[0]  - post handle...


In [14]:
stepsize = yml_data["yolov5"]["topView"]["stepsize"]

# 2. Create img from CSF
non_ground_img = pcd2img_np(non_grd,"z",stepsize)

# 3. Get min
minimum, maximum = non_grd.get_min_bound(), non_grd.get_max_bound()

coor_tape = np.asarray([v3.x,v3.y,v3.tape]).transpose()
coor_tape = scale_coord(coor_tape, (1/stepsize,1/stepsize), (-minimum[0]/stepsize,maximum[1]/stepsize))

coor_dickson = np.asarray([v3.x,v3.y,v3.dickson_lidar]).transpose()
coor_dickson = scale_coord(coor_dickson, (1/stepsize,1/stepsize), (-minimum[0]/stepsize,maximum[1]/stepsize))

coor_diff = np.asarray([v3.x,v3.y,v3.tape-v3.dickson_lidar]).transpose()
coor_diff = scale_coord(coor_diff, (1/stepsize,1/stepsize), (-minimum[0]/stepsize,maximum[1]/stepsize))

### On my generated CSV before Raj

In [21]:
new_pd =  pd.read_csv('output/Div02k_02/Div02k_02.csv')
print(new_pd)
#new_pd.y = -new_pd.y
new_pd = np.asarray([new_pd.x, new_pd.y, new_pd.h]).transpose()
new_pd = scale_coord(new_pd, (1/stepsize, 1/stepsize), (-minimum[0]/stepsize,maximum[1]/stepsize))

print(new_pd.shape)

     Unnamed: 0          x           y          h
0             0 -33.380821   72.467926  10.289234
1             1 -82.062881   74.317215   7.191938
2             2 -89.110291   58.373257   9.413141
3             3 -69.867378   82.913956   7.962500
4             4 -76.814819   67.469818   7.809000
..          ...        ...         ...        ...
284         284  -3.841705  159.584915   8.153703
285         285   7.004303  145.990067   6.940000
286         286   3.505585  138.542877   7.476000
287         287  41.691528  188.473969   6.277500
288         288  59.884827  151.637924   9.245531

[289 rows x 4 columns]
(289, 3)


In [22]:
def draw_height_on_coord(img, x_y_h:np.ndarray, on_left:str="tape"): #(xc,yc,confidence,label)
    img = img.copy()
    if not x_y_h.size:
        return img
    red = (255,0,0)
    color = red
    font = cv2.FONT_HERSHEY_SIMPLEX
    for stuff in x_y_h:
        x,y,h = int(stuff[0]), int(stuff[1]), stuff[2]
        img = cv2.circle(img,(x,y), 1, color,5)
        img = cv2.putText(img, f"{h:.1f}", (x,y),font, 1,red,2,cv2.LINE_AA)
        
    # Writing image name
    img = cv2.putText(img, on_left, (50,50), font, 2, red, 2, cv2.LINE_AA)
    return img

img_tape = draw_height_on_coord(non_ground_img, coor_tape,"tape")
#img_dickson = draw_height_on_coord(non_ground_img, coor_dickson,"dickson")
img_diff = draw_height_on_coord(non_ground_img, coor_diff,"dickson")

cv2.imwrite(f"datasets/img_tape.png", img_tape)
#cv2.imwrite(f"datasets/img_dickson.png", img_dickson)
cv2.imwrite(f"datasets/img_diff.png", img_diff)

img_dickson = draw_height_on_coord(non_ground_img, new_pd,"dickson")
cv2.imwrite(f"datasets/img_dickson.png", img_dickson)


True